In [1]:
import pandas as pd
from darklyrics import get_band_lyrics
from metallum import get_band_info
from utils import get_genres
from lyricsutils import band_word_count

In [2]:
letter = 'C'
bands = pd.read_csv('bands/bands' + letter + '.csv').set_index('id', drop=True)
ids = pd.read_csv('ids/ids' + letter + '.csv').set_index('name', drop=True)

In [3]:
import pandas as pd
import re
from darklyrics import get_band_lyrics
from utils import get_genres
from lyricsutils import word_count, strip_lyrics

class Album(object):
    def __init__(self, name, lyrics):
        self.name = name
        self.lyrics = lyrics
    
    @property
    def song_names(self):
        a = list(self.lyrics.keys())
        return [b[b.index('. ')+2:] for b in a]

class Band(object):
    def __init__(self, name, album_dict, country=None, genre=None):
        self.name = name
        self.albums = [Album(key, val) for key, val in album_dict.items()]
        self.country = country
        self._genre = genre
    
    @classmethod
    def from_df(cls, df):
        lyrics = get_band_lyrics(df['name'])
        print(lyrics)
        return cls(df['name'], lyrics, country=df['country'], genre=df['genre'])
    
    @property
    def album_names(self):
        return [album.name for album in self.albums]
    
    @property
    def genre(self):
        pattern = '[a-zA-Z]+'
        genre_list = [match.lower() for match in re.findall(pattern, self._genre)]
        genre_list = sorted(set(genre_list))
        if 'metal' in genre_list:
            genre_list.remove('metal')
        return genre_list
    
    def get_word_counts(self):
        if len(self.albums) == 0:
            return pd.DataFrame()
        df_list = []
        for album in self.albums:
            for song_name, lyrics in zip(*(album.song_names, album.lyrics.values())):
                word_count_dict = word_count(strip_lyrics(lyrics))
                song_df = pd.DataFrame.from_dict(word_count_dict, orient='index')
                if len(song_df) != 0:
                    song_df.columns = [song_name]
                    df_list.append(song_df)
        df = pd.concat(df_list, axis=1).fillna(0)
        return df
    
    def get_word_means(self):
        df = self.get_word_counts()
        return df.mean(axis=1).sort_values(ascending=False)

In [4]:
band_name = 'Cynic'
band_df = bands.loc[ids.loc[band_name].id]
band = Band.from_df(band_df)

{'Reflections Of A Dying World': {}, 'Demo': {}, 'Focus': {}, 'Traced In Air': {}, 'Re-Traced': {}, 'Carbon-Based Anatomy': {}, 'Kindly Bent To Free Us': {}}


In [144]:
word_means_list = []
for band_name in bands['name'][bands['genre'].str.contains('Black/Death/Thrash')].values:
    band_df = bands.loc[ids.loc[band_name].id]
    band = Band.from_df(band_df)
    word_means_list.append(band.get_word_means())

url not found: http://www.darklyrics.com/c/calvario.html


AttributeError: 'list' object has no attribute 'items'

In [147]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 5))
band_name = 'Cynic'
band_df = bands.loc[ids.loc[band_name].id]
band = Band.from_df(band_df)
word_means = band.get_word_means()[:20]
sns.barplot(x=word_means.index, y=word_means.values, hue=[0]*len(word_means))

plt.figure(figsize=(20, 5))
band_name = 'Cannibal Corpse'
band_df = bands.loc[ids.loc[band_name].id]
band = Band.from_df(band_df)
word_means = band.get_word_means()[:20]
sns.barplot(x=word_means.index, y=word_means.values, hue=[0]*len(word_means))
plt.show()

{'Reflections Of A Dying World': {}, 'Demo': {}, 'Focus': {}, 'Traced In Air': {}, 'Re-Traced': {}, 'Carbon-Based Anatomy': {}, 'Kindly Bent To Free Us': {}}


ValueError: No objects to concatenate